In [5]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

^C


In [6]:
import pandas as pd
import torch

from sentence_transformers import SentenceTransformer, losses, models
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from torch import nn

Create drug-pairs dataset

In [2]:
df = pd.read_json("processed.json")
df.drop(columns=['DRUG_CID', "DRUG_NAME"], inplace=True)
df['INDEX'] = df.index
df1 = df.copy()
df2 = df.copy()
df1.columns = ['DRUG_ID1', 'INTERACTIONS1', 'STRUCTURE1', 'TARGET1', 'ENZYME1', 'PATH1', 'SIDE_EFFECTS1', 'INDICATIONS1', 'OFFSIDE_EFFECT1', 'INDEX1']
df2.columns = ['DRUG_ID2', 'INTERACTIONS2', 'STRUCTURE2', 'TARGET2', 'ENZYME2', 'PATH2', 'SIDE_EFFECTS2', 'INDICATIONS2', 'OFFSIDE_EFFECT2', 'INDEX2']
df = pd.merge(df1, df2, how="cross")
df = df[df['DRUG_ID1'] != df['DRUG_ID2']]
df.reset_index(inplace=True)
df['INTERACTION'] = df.apply(lambda row: row['INTERACTIONS1'][row['INDEX2']], axis=1)

df['id_pair'] = df[['DRUG_ID1', 'DRUG_ID2']].apply(lambda x: '-'.join(sorted(map(str, x))), axis=1)
df.drop_duplicates(subset='id_pair', inplace=True)
df.dropna(inplace=True)
df.drop(columns=["index", 'INTERACTIONS1', 'INTERACTIONS2', 'INDEX1', 'INDEX2', 'id_pair'], inplace=True)

99235

In [ ]:
data = {}
to_edit = ['STRUCTURE', 'TARGET', 'ENZYME', 'PATH', 'SIDE_EFFECTS', 'INDICATIONS', 'OFFSIDE_EFFECT']
for column in to_edit:
    column1 = column + '1'
    column2 = column + '2'
    data[column] = [InputExample(texts=[" ".join(map(str, row[0])), " ".join(map(str, row[1]))], label=row[2])
                    for row in zip(df[column1], df[column2], df['INTERACTION'])]

In [3]:
word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(data['STRUCTURE'], shuffle=True, batch_size=2)
train_loss = losses.ContrastiveLoss(model=model)

model.fit(
    [(train_dataloader, train_loss)],
    epochs=10,
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49618 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:

print(torch.cuda.is_available())

False
